# Using peptdeep for MHC class I immunopeptidomics

This notebook introduces how to generate spectral libraries for immunopeptidomics analysis from a list of protein sequences. This entails several steps:

1. unspecific digestion of protein sequences
2. selection of peptide sequences used for library prediction by peptdeep-hla predicition
   2.1 using the pretrained model
   2.2 using an improved model by including a transfer learning step
3. spectral library prediction
4. matching the peptides back to the proteins (this can be done before or after library prediction or seach)  



Note that pydivsufsort package is not installed by peptdeep by default. Install by:
```
pip install "peptdeep[development,hla]"
```

Or install within jupyter notebook:

In [37]:
%pip install -q pydivsufsort

Note: you may need to restart the kernel to use updated packages.


## 1. Unspecific digestion in alphabase

The unspecific digestion workflow uses the longest common prefix (LCP) algorithm, which is based on suffix array data structure, has been proven to be very efficient for unspecific digestion [https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-11-577]. Here we used `pydivsufsort`, a Python wrapper of a high-performance C library libdivsufsort [https://github.com/y-256/libdivsufsort], to facilitate LCP-based digestion.

This means, the digestion is performed on a single sequence of strings and retrives both the peptide sequence as well as the start and stop indeces of the peptide within the complete sequence. Therefore, unspecific digestion in alphabase involves two steps:

1. concatenation of protein sequences into a single sequence
2. unspecific digestion



#### 1.1 Concatenate protein sequences into a single sequence

The protein sequences are concatenated into a single sequence. The sequences are seperated by a sentinel character, in this case '$', so that no peptides across proteins are formed. Note that the first and last sentinel characters are crutial as well.


In [38]:
def concat_sequences_for_nonspecific_digestion(seq_list, sep="$"):
    return sep + sep.join(seq_list) + sep

In [39]:
prot_seq_list = ["MABCDEKFGHIJKLMNOPQRST","FGHIJKLMNOPQR"]
cat_prot = concat_sequences_for_nonspecific_digestion(prot_seq_list, sep="$")
cat_prot

'$MABCDEKFGHIJKLMNOPQRST$FGHIJKLMNOPQR$'

The same can be done directly from a fasta: 
@ Feng do you have an example fasta somwhere? 

In [40]:
from peptdeep.hla.hla_utils import load_prot_df
fasta = load_prot_df(r"D:\Software\FASTA\Human\example.fasta")
fasta

,protein_id,full_name,gene_name,description,sequence,nAA
tr|A0A024R161|A0A024R161_HUMAN,A0A024R161,tr|A0A024R161|A0A024R161_HUMAN,DNAJC25-GNG10,tr|A0A024R161|A0A024R161_HUMAN Guanine nucleot...,MGAPLLSPGWGAGAAGRRWWMLLAPLLPALLLVRPAGALVEGLYCG...,153
tr|A0A024RAP8|A0A024RAP8_HUMAN,A0A024RAP8,tr|A0A024RAP8|A0A024RAP8_HUMAN,KLRC4-KLRK1,"tr|A0A024RAP8|A0A024RAP8_HUMAN HCG2009644, iso...",MGWIRGRRSRHSWEMSEFHNYNLDLKKSDFSTRWQKQRCPVVKSKC...,216


In [41]:
from peptdeep.hla.hla_utils import cat_proteins
cat_fasta = cat_proteins(fasta['sequence'])
cat_fasta

'$MGAPLLSPGWGAGAAGRRWWMLLAPLLPALLLVRPAGALVEGLYCGTRDCYEVLGVSRSAGKAEIARAYRQLARRYHPDRYRPQPGDEGPGRTPQSAEEAFLLVATAYETLKVSQAAAELQQYCMQNACKDALLVGVPAGSNPFREPRSCALL$MGWIRGRRSRHSWEMSEFHNYNLDLKKSDFSTRWQKQRCPVVKSKCRENASPFFFCCFIAVAMGIRFIIMVTIWSAVFLNSLFNQEVQIPLTESYCGPCPKNWICYKNNCYQFFDESKNWYESQASCMSQNASLLKVYSKEDQDLLKLVKSYHWMGLVHIPTNGSWQWEDGSILSPNLLTIIEMQKGDCALYASSFKGYIENCSTPNTYICMQRTV$'

#### 1.2 Unspecific digestion

Use `alphabase.protein.lcp_digest.get_substring_indices` to get all non-redundant non-specific peptide sequences from the concatenated protein sequence. The digested peptide sequences are stored in a dataframe based on their start and stop indices in the concantenated protein sequence string. To save the RAM, the `peptdeep.hla` module works on start and stop indices instead of on peptide sequences directly. This will save about 8 times of the RAM for HLA-I peptides (length from 7 to 14, deomnstrated below). For a large protein sequence database, there will be millions of unspecific peptides, so working with strings is not feasible for a complete human fasta due to the requirements of extremely large RAM. (~ 70M unspecific sequences from the reviewed swissprot fasta require ~ 4-5 GB RAM already).

Using the get_substring_indices function we extract the start and stop indices of all peptide sequences between 7 and 14 aa (min_len, max_len) from the concatenated protein sequences. All peptides sequences are unique, guranteed by the LCP algorithm.

In [42]:
from alphabase.protein.lcp_digest import get_substring_indices
import pandas as pd
import sys

start_idxes, stop_idxes = get_substring_indices(
    cat_fasta, min_len=8, max_len=14, stop_char="$"
)
digest_pos_df = pd.DataFrame({
    "start_pos": start_idxes,
    "stop_pos": stop_idxes,
})
digest_pos_df

,start_pos,stop_pos
0,1,9
1,1,10
2,1,11
3,1,12
4,1,13
...,...,...
2438,361,370
2439,361,371
2440,362,370
2441,362,371


In [43]:
RAM_use_idxes = sys.getsizeof(digest_pos_df)*1e-6

The unspecific peptide sequences can be localted by the `start_pos` and `stop_pos`.

In [44]:
digest_pos_df["sequence"] = digest_pos_df[
    ["start_pos","stop_pos"]
].apply(lambda x: cat_fasta[slice(*x)], axis=1)
digest_pos_df

,start_pos,stop_pos,sequence
0,1,9,MGAPLLSP
1,1,10,MGAPLLSPG
2,1,11,MGAPLLSPGW
3,1,12,MGAPLLSPGWG
4,1,13,MGAPLLSPGWGA
...,...,...,...
2438,361,370,NTYICMQRT
2439,361,371,NTYICMQRTV
2440,362,370,TYICMQRT
2441,362,371,TYICMQRTV


In [45]:
RAM_use_seqs = sys.getsizeof(digest_pos_df["sequence"])*1e-6

In [46]:
f"seq RAM = {RAM_use_seqs:.5f} Mb, idxes RAM = {RAM_use_idxes:.5f}, ratio = {RAM_use_seqs/RAM_use_idxes:.5f}"

'seq RAM = 0.16621 Mb, idxes RAM = 0.01969, ratio = 8.44230'

## Selection of peptide sequences used for library prediction
The digest_prot_df contains all unspecifically digested peptide sequences between 7 and 14 aa generatable from the concatenated protein sequences. This list is reduced using a HLA1_Binding_Classifier from peptdeep.hla.hla_class1. Two different model architectures are available, an LSTM model (HLA_Class_I_LSTM) and a BERT model (HLA_Class_I_BERT). A pretrained model is only available for the LSTM model architecture.
The HLA1_Binding_Classifer can be used with a pretrained model, tuned with existing peptide data or trained from scratch. Training of a new model should be considered carefully and will not be covered in this tutorial.
   

### 2.2 Selection of peptide seqeuence candidates without transferlearning

Selection of peptide sequences for library predicition using the pretrained model can be done in a few steps. First, the Classifier model needs to be initialized and the pretrained model is loaded. Next, we can use any kind of dataframe containing peptide sequences to predict how likely there are HLA peptides, the only requirement beeing that the column containing the peptides is called 'sequence'.


In [47]:
from peptdeep.hla.hla_class1 import HLA1_Binding_Classifier

model = HLA1_Binding_Classifier()
model.load_pretrained_hla_model()
manual_prediction = model.predict(digest_pos_df)
manual_prediction


,start_pos,stop_pos,sequence,nAA,HLA_prob_pred
0,1,9,MGAPLLSP,8,0.239477
1,145,153,REPRSCAL,8,0.061692
2,146,154,EPRSCALL,8,0.137313
3,155,163,MGWIRGRR,8,0.056462
4,156,164,GWIRGRRS,8,0.001298
...,...,...,...,...,...
2438,112,126,KVSQAAAELQQYCM,14,0.243115
2439,317,331,NGSWQWEDGSILSP,14,0.021114
2440,79,93,DRYRPQPGDEGPGR,14,0.060635
2441,113,127,VSQAAAELQQYCMQ,14,0.355900


Next, we can filter the list based on the HLA_prob_pred. The higher the probability, the more likely it is for the peptide sequence to be present in a immunopeptidomics sample. It is not recommended to use a cut-off below 0.7 as this inflates the spectral library massively. It is rather recommended to use more conservative cut-offs. 

In [48]:
manual_prediction[manual_prediction['HLA_prob_pred'] > 0.7]

,start_pos,stop_pos,sequence,nAA,HLA_prob_pred
17,168,176,EMSEFHNY,8,0.793702
24,130,138,KDALLVGV,8,0.817415
31,137,145,VPAGSNPF,8,0.751329
37,170,178,SEFHNYNL,8,0.940019
67,181,189,KSDFSTRW,8,0.895964
...,...,...,...,...,...
2318,95,109,QSAEEAFLLVATAY,14,0.969541
2378,329,343,SPNLLTIIEMQKGD,14,0.756001
2382,5,19,LLSPGWGAGAAGRR,14,0.733784
2408,110,124,TLKVSQAAAELQQY,14,0.891976


As described above, directly using the sequences for classification can be memory intense for large lists of sequences. Thereby, the manual concatenation, unspecific digestion, predicition and filtering is only suggested for small sets of proteins or integration of selected sequences (e.g mutations, nuORFs etc.). This can be circumvented by directly predicting and filtering from a fasta using model.predict_from_proteins(). This executes the concatenation, unspecific digestion, predicition and filtering automatically in batches. Thereby the whole process can be done more efficient and be performed without a specialized computation infrastructure.

In [49]:
model.predict_from_proteins(fasta, prob_threshold=0.7)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.24it/s]


,start_pos,stop_pos,nAA,HLA_prob_pred,sequence
0,168,176,8,0.793702,EMSEFHNY
1,130,138,8,0.817415,KDALLVGV
2,137,145,8,0.751329,VPAGSNPF
3,170,178,8,0.940019,SEFHNYNL
4,181,189,8,0.895964,KSDFSTRW
...,...,...,...,...,...
143,95,109,14,0.969541,QSAEEAFLLVATAY
144,329,343,14,0.756001,SPNLLTIIEMQKGD
145,5,19,14,0.733784,LLSPGWGAGAAGRR
146,110,124,14,0.891976,TLKVSQAAAELQQY


### 2.2 Selection of peptide seqeuence candidates with transferlearning

To perform transferlearning we need a list of peptide sequences we expect to be present in our sample. These peptides can be retrived from several different sources like DDA or directDIA search results. It is recommended to use at the very least 1000 sequences for transferlearning. The more sequences available the better the transferlearning step works. The model performance can be assessed after transferlearning and should be assessed before predicition. 

First, the Classifier model needs to be initialized and the pretrained model is loaded. Next, a protein dataframe is added, in this example the previousely loaded fasta file. The protein dataframe is used by the Classifier internaly to draw negative training data during model training and testing.

In [50]:
model = HLA1_Binding_Classifier()
model.load_pretrained_hla_model()
model.load_proteins(fasta)

Next, we load the peptide sequences wee use for transferlearning and split it into a training and testing dataset. This step is very important to assess the model performance after transferlearning. Here, we use the digest_pos_df generated above. As these are no immunopeptides, but a list of unspecifically digested proteins, the model performance will not improve, but the pronciples remain the same.  
@ Feng should we include a example file so that the model is actually improved or just use this? 

In [51]:
test_seq_df = digest_pos_df.sample(frac=0.2)
train_seq_df = digest_pos_df.drop(index=test_seq_df.index)
len(train_seq_df), len(test_seq_df)

(1954, 489)

Now, we train the model using the training sequence dataframe. In this example we use 10 training epochs, in a real experiment more should be used. Good starting points are 40 epochs for a training dataset of around 10000 sequences or 100 epochs for a training dataset of around 1000 sequences. For a real experiment the warmup_epochs can be increased to 10.  

In [52]:

model.train(train_seq_df,
            epoch=10, warmup_epoch=5, 
            verbose=True)

2024-07-18 10:24:25> Training with fixed sequence length: 0


c:\Users\wahle\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Training] Epoch=1, lr=2e-05, loss=1.4192258289882116


c:\Users\wahle\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Training] Epoch=2, lr=4e-05, loss=1.0882413131850106


...

c:\Users\wahle\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Training] Epoch=8, lr=6.545084971874738e-05, loss=0.6962822931153434


c:\Users\wahle\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Training] Epoch=9, lr=3.4549150281252636e-05, loss=0.6965692894799369
[Training] Epoch=10, lr=9.549150281252633e-06, loss=0.6948717491967338


c:\Users\wahle\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


We can assess the model performance after transferlearning using the model.test() function on the training and testing data. This can also be done before transferlearning to assess how well the model fits the available data already. The test assesses the precision, recall and fals positive rate of the model at different probability cut offs. As a rule of thumb a false postitve rate above 7% (@FENG adjust in case lower/higher) is not recomendable because the peptide list gets disproportionally larger, leading to lower IDs during the search. In case of a high false postitive rate, the probability cut off at which the peptides are predicted should be increased.  

In [53]:
model.test(train_seq_df)

,HLA_prob_pred,precision,recall,false_positive
0,0.5,0.507442,0.558342,0.541965
1,0.6,0.516129,0.016377,0.015353
2,0.7,NaN,0.000000,0.000000
3,0.8,NaN,0.000000,0.000000
4,0.9,NaN,0.000000,0.000000


In [54]:
model.test(test_seq_df)

,HLA_prob_pred,precision,recall,false_positive
0,0.5,0.47807,0.445808,0.486708
1,0.6,0.62500,0.020450,0.012270
2,0.7,1.00000,0.002045,0.000000
3,0.8,NaN,0.000000,0.000000
4,0.9,NaN,0.000000,0.000000


After transferlearning and testing the new model, peptides can be predicted as with the pretrained model. 

In [55]:
model.predict_from_proteins(digest_pos_df)

100%|██████████| 1/1 [00:00<00:00,  1.24it/s]


,start_pos,stop_pos,nAA,HLA_prob_pred,sequence
0,26801,26809,8,0.715877,SEFHNYNL


## Spectral library prediciton